In [11]:
#Image encoding tests
#FRQI
from qiskit import QuantumCircuit,Aer, transpile
from qiskit.circuit.library.standard_gates import RYGate
from qiskit.circuit import Parameter, ParameterVector
import math
import matplotlib.pyplot as plt
import numpy as np
from qiskit.visualization import plot_histogram

def cry(param,amount_control):
    #Return a c..cry gate, with amount_control control and the parameter param binded to the ry gate
    return RYGate(param*2).control(amount_control)

def prepare_img(img):
    #Scale the image to match the requirement of radians
    return(np.array(img)*(math.pi/2)/255)
def prepare_frqi(params):
    #Return the frqi circuit for the ParameterVector params
    to_prepare = len(params)
    max_size = len(format(to_prepare-1,"b"))
    binary = [format(i,"b").zfill(max_size) for i in range(to_prepare)]
    qc = QuantumCircuit(max_size+1)
    qc.h(range(max_size))
#     qc.barrier()
    last_b = '0'*max_size
    for b in binary:
        x_loc = []
        for ind in range(len(b)):
            if b[ind] != last_b[ind]:
                x_loc.append(ind)
        last_b = b
        if len(x_loc) != 0:
            qc.x(x_loc)
        qc.append(cry(params[int(b,2)],max_size),range(max_size+1))
#         qc.barrier()
    return qc
print("Started generation")
img_size = 32
array = np.zeros((img_size,img_size))
for i in range(img_size):
    array[i,i] = 255
print("Image is being prepared")
image = prepare_img(array)
# image = prepare_img([[255,0],[0,255]])
print("Launched FRQI preparation")
frqi = prepare_frqi(ParameterVector('θ',image.shape[0]*image[0].shape[0]))

Started generation
Image is being prepared
Launched FRQI preparation


In [12]:
print("Depth :",frqi.depth())
print("Ops : ",frqi.count_ops())
print("Num qubits : ",frqi.num_qubits)

Depth : 2048
Ops :  OrderedDict([('x', 2036), ('c10ry', 1024), ('h', 10)])
Num qubits :  11


In [2]:
# print("image = ",image)
size = (len(image),len(image[0]))
image_list = np.array(image).reshape(len(image)*len(image[0]))
# plt.imshow(image_list.reshape(size))
# plt.show()
print("Binded the arguments")
binded = frqi.bind_parameters(image_list)
# binded.x(range(3))
binded.measure_all()
print("Starting execution")
backend = Aer.get_backend("qasm_simulator")
transpiled = transpile(binded,backend)
shots = 8192
job = backend.run(transpiled, shots = shots)
# display(binded.draw(output = 'mpl'))
# display(plot_histogram(job.result().get_counts()))
print(dict(job.result().get_counts()))

Started generation
Image is being prepared
Launched FRQI preparation


KeyboardInterrupt: 

In [ ]:
import math
def retrieve_image_from_count(count, shots, image_width):
    #Divide the values by the number of shots to find an approximation of the sin and cos of the angle 
    normalized = {key:values/shots for (key,values) in count.items()}
    bit_size = len(list(normalized.keys())[0])
    #Binary list with the right side of the bit representation (missing the leftmost bit)
    partial_bin_list = [bin(ent)[2:].zfill(bit_size-1) for ent in range(image_width**2)]
    full_bin_list = []
    
    #Adding the leftmost bit (which determine if we have the cos or the sin of the angle)
    for b in partial_bin_list:
        full_bin_list.append('0'+b)
        full_bin_list.append('1'+b)
    for key in full_bin_list:
        if key not in normalized :
            normalized[key] = 0
    m = max(normalized.values())
    for key,value in normalized.items():
        normalized[key] = value/m
    for key,value in normalized.items():
        if key[0] == '0':    
            normalized[key] = math.acos(value)
        else : 
            normalized[key] = math.asin(value)
    normalized = {k:v for (k,v) in sorted(normalized.items(), key = lambda item : int(item[0][1:],2)+0.5*int(item[0][0],2) )}
    val = list(normalized.values())
    final_img = []
    for i in range(0,len(val),2):
        final_img.append((val[i]+val[i+1])/2)
    print(len(final_img))
    return(final_img)
        
    
result = job.result().get_counts()
img = retrieve_image_from_count(result, shots,4)
plt.imshow(np.array(img).reshape(4,4))
plt.show()